# Important types for sequences

During the sequence tutorials we will mostly work using sequences of words. 

It might be useful then to define a type that will be used in the different assignemnts that you will have to face.

In [2]:
workspace()

type sequence
    words::Array{String}
    labels::Array{String}
    
    function sequence(words)
        states=["*" for x in words]
        return new(words, states)
    end
    
    function sequence(words, states)
        return new(words, states)
    end
    
end

In [3]:
seq1 = sequence(["the", "house", "is", "big"])

sequence(String["the","house","is","big"],String["*","*","*","*"])

In [4]:
seq1.labels

4-element Array{String,1}:
 "*"
 "*"
 "*"
 "*"

In [5]:
length(seq1.words)

4

### Define a sequence of words with a sequence of labels

In [6]:
seq2 = sequence(["the", "house", "is", "big"],
                ["det","noun","verb","adj"])

sequence(String["the","house","is","big"],String["det","noun","verb","adj"])

In [7]:
seq2.labels

4-element Array{String,1}:
 "det" 
 "noun"
 "verb"
 "adj" 

In [8]:
seq = sequence(["the", "house", "is", "big"], 
               ["det","noun","verb","adj"])

sequence(String["the","house","is","big"],String["det","noun","verb","adj"])

In [9]:
seq.labels

4-element Array{String,1}:
 "det" 
 "noun"
 "verb"
 "adj" 

#### Be carefull with phrases

The type that we just defined does not accept single strings

In [10]:
sequence("the house is big")

LoadError: LoadError: MethodError: Cannot `convert` an object of type String to an object of type Array{String,N}
This may have arisen from a call to the constructor Array{String,N}(...),
since type constructors fall back to convert methods.
while loading In[10], in expression starting on line 1

Nevertheless the ```sequence``` type can accept an Array with a single string containing a sequence. 

This is a behaviour we might not want it

In [11]:
seq2 = sequence(["the house is big"])

sequence(String["the house is big"],String["*"])

In [12]:
seq2.words

1-element Array{String,1}:
 "the house is big"

In [13]:
split("the house is big")

4-element Array{SubString{String},1}:
 "the"  
 "house"
 "is"   
 "big"  

In [14]:
split("the house is big"," ")

4-element Array{SubString{String},1}:
 "the"  
 "house"
 "is"   
 "big"  

### Rainy sunny example

In [15]:
Sigma = ["walk", "shop", "clean", "tennis"]
Lambda = ["rainy", "sunny"]

sequence_list = []

s1 = sequence(["walk", "walk", "shop", "clean"],
             ["rainy", "sunny", "sunny", "sunny"])

s2 = sequence(["walk", "walk", "shop", "clean"], 
              ["rainy", "rainy", "rainy", "sunny"])

s3 = sequence(["walk", "shop", "shop", "clean"], 
              ["sunny", "sunny", "sunny", "sunny"])

train_sequences = [s1, s2, s3]

s1_t = sequence(["walk", "walk", "shop", "clean"], 
                ["rainy", "sunny", "sunny", "sunny"])

s2_t = sequence(["clean", "walk", "tennis", "walk"], 
                ["sunny", "sunny", "sunny", "sunny"])

test_sequences = [s1_t, s2_t];

In [16]:
train_sequences[2]

sequence(String["walk","walk","shop","clean"],String["rainy","rainy","rainy","sunny"])

# Hidden markov model

In [17]:
Dict{String,Float64}(["lala"=>34])

Dict{String,Float64} with 1 entry:
  "lala" => 34.0

In [18]:
train_sequences

3-element Array{sequence,1}:
 sequence(String["walk","walk","shop","clean"],String["rainy","sunny","sunny","sunny"])
 sequence(String["walk","walk","shop","clean"],String["rainy","rainy","rainy","sunny"])
 sequence(String["walk","shop","shop","clean"],String["sunny","sunny","sunny","sunny"])

In [19]:
test_sequences

2-element Array{sequence,1}:
 sequence(String["walk","walk","shop","clean"],String["rainy","sunny","sunny","sunny"])  
 sequence(String["clean","walk","tennis","walk"],String["sunny","sunny","sunny","sunny"])

#### Get all possible states and all possible words

In [20]:
word_to_int = Dict{String,Int64}()
state_to_int = Dict{String,Int64}()

Dict{String,Int64} with 0 entries

In [21]:
function get_possible_words_and_states(sequences)
    state_counter = 1
    word_counter = 1
    
    possible_words = Set{String}()
    possible_states = Set{String}()
    
    for seq in sequences
        for (t,w) in zip(seq.labels, seq.words)
            push!(possible_states, t)
            push!(possible_words, w)
        end
    end
    
    return possible_words, possible_states
end

get_possible_words_and_states (generic function with 1 method)

In [22]:
possible_words, possible_states = get_possible_words_and_states([train_sequences; test_sequences])

(Set(String["tennis","walk","clean","shop"]),Set(String["sunny","rainy"]))

#### map words to positions and states to positions

In [23]:
num_words = length(possible_words)
num_states = length(possible_states)

2

In [24]:
function assign_elements_to_integers(elements)
    element_to_pos = Dict{String, Int64}()
    for (k,e) in enumerate(elements)
        element_to_pos[e] = k
    end
    return element_to_pos
end

assign_elements_to_integers (generic function with 1 method)

In [25]:
function assign_elements_to_integers2(elements)
    element_to_pos = Dict{String, Int64}()
    k = 1
    for e in elements
        element_to_pos[e] = k
        k +=1
    end
    return element_to_pos
end

assign_elements_to_integers2 (generic function with 1 method)

In [26]:
word_to_pos = assign_elements_to_integers(possible_words);
state_to_pos = assign_elements_to_integers(possible_states);

### Computing  sufficient statistics (counts) of the HMM

In [27]:

function update_initial_counts!(initial_counts, seq, state_to_pos)
    initial_counts[state_to_pos[seq.labels[1]]] = initial_counts[state_to_pos[seq.labels[1]]] + 1
end

update_initial_counts! (generic function with 1 method)

In [28]:
function update_transition_counts!(transition_counts, seq, state_to_pos)
    for (t1,t2) in zip(seq.labels[1:end-1], seq.labels[2:end])
        transition_counts[state_to_pos[t1], state_to_pos[t2]] += 1 
    end
    
end

update_transition_counts! (generic function with 1 method)

In [29]:
function update_emission_counts!(emission_counts, seq, state_to_pos, word_to_pos)
   
    for (t,w) in zip(seq.labels, seq.words)
        emission_counts[state_to_pos[t], word_to_pos[w]] += 1 
    end
    
end

update_emission_counts! (generic function with 1 method)

In [30]:
function update_final_counts!(final_counts, seq, state_to_pos)
    final_counts[state_to_pos[seq.labels[end]]] +=1
end

update_final_counts! (generic function with 1 method)

In [31]:
function sufficient_statistics_hmm(sequences, state_to_pos, word_to_pos)
    
    n_states = length(state_to_pos)
    n_words = length(word_to_pos)
    
    initial_counts     = zeros(n_states)
    transition_counts = zeros(n_states, n_states)
    final_counts       = zeros(n_states) 
    emission_counts     = zeros(n_states, n_words)
    
    for seq in sequences
        update_initial_counts!(initial_counts, seq, state_to_pos)
        update_transition_counts!(transition_counts, seq,  state_to_pos)
        update_emission_counts!(emission_counts, seq,  state_to_pos, word_to_pos) 
        update_final_counts!(final_counts, seq,  state_to_pos) 
    end
    
    return initial_counts, transition_counts, final_counts, emission_counts
    
end

sufficient_statistics_hmm (generic function with 1 method)

In [32]:
counts = sufficient_statistics_hmm(train_sequences, state_to_pos, word_to_pos);

In [33]:
initial_counts, transition_counts, final_counts, emission_counts = counts;

In [34]:
initial_counts

2-element Array{Float64,1}:
 1.0
 2.0

In [35]:
transition_counts

2×2 Array{Float64,2}:
 5.0  0.0
 2.0  2.0

In [36]:
final_counts

2-element Array{Float64,1}:
 3.0
 0.0

In [37]:
emission_counts

2×4 Array{Float64,2}:
 0.0  2.0  3.0  3.0
 0.0  3.0  0.0  1.0

#### Sanity Checks HMM

- Initial counts must sum to the number of sentences  $$ \sum_{k=1}^K C_{\text{init}}(c_k) = M$$

- Transition counts and Final Counts should sum to the number of tokens: $$\sum_{k,l=1}^K C_{\text{trans}}(c_k,c_l)  + \sum_{k=1}^K C_{\text{final}}(c_k) = M \cdot N$$

- Emission counts must sum to the number of tokens
$$
\sum_{j=1}^J \sum_{k=1}^K C_{\text{emiss}}(w_j,c_k) = M \cdot N 
$$

In [38]:
emission_counts

2×4 Array{Float64,2}:
 0.0  2.0  3.0  3.0
 0.0  3.0  0.0  1.0

In [39]:
M = length(train_sequences)
N = length(train_sequences[1].words)
print("M: ", M, "\n","N: ", N,"\n" ,"M*N: ", M*N)

M: 3
N: 4
M*N: 12

In [40]:
print("\ninitial_counts sum: ", sum(initial_counts))
print("\nemission_counts sum: ", sum(emission_counts))
print("\ntransition and final counts sum: ", sum(transition_counts) + sum(final_counts))


initial_counts sum: 3.0
emission_counts sum: 12.0
transition and final counts sum: 12.0

In [41]:
function check_counts(data, 
                      possible_states,
                      initial_counts,
                      transition_counts, 
                      emission_counts, 
                      final_counts)
    """
    This Check is only valid if all instances have the same length!!!!
    """
    n_samples = length(data)
    sequence_length = length(data[1].words)
    problem_checks = []
    
    if sum(initial_counts) != n_samples
        print("\nERROR: initial_counts are not correctly computed")
        push!(problem_checks,"initial_counts")
    end
    
    if sum(transition_counts) + sum(final_counts) != sequence_length*n_samples
        print("\nERROR: transition_counts are not correctly computed")
        push!(problem_checks,"transition_counts")
    end
    
    if sum(emission_counts)  != sequence_length*n_samples
        print("\nERROR: emission_counts are not correctly computed")
        push!(problem_checks,"emission_counts")
    end
    
    if length(problem_checks) ==0
        print("\nAll checks passed")
    end
end


check_counts (generic function with 1 method)

In [42]:
 check_counts(train_sequences, 
             possible_states,
             initial_counts,
             transition_counts, 
             emission_counts, 
             final_counts)


All checks passed

## From counts to probabilities

The following formulas specify how to find the parameters of the HMM:

$$
P_{\text{init}}(c_k \,\vert\, \text{start}) = \frac{C_{\text{init}}(c_k)}{ \sum_{k=1}^K
C_{\text{init}} (c_l)}
$$

$$
P_{\text{final}}(\text{stop} \,\vert\, c_l) = \frac{C_{\text{final}}(c_l) }
{\sum_{k=1}^K C_{\text{trans}}(c_k,c_l) + C_{\text{final}}(c_l)}
$$

$$
P_{\text{trans}}( c_k \,\vert\, c_l) = \frac{C_{\text{trans}}(c_k, c_l) }
{\sum_{p=1}^K C_{\text{trans}}(c_p,c_l) + C_{\text{final}}(c_l)}
$$

$$
P_{\text{emiss}} (w_j \,\vert\, c_k) = \frac{C_{\text{emiss}} (w_j, c_k) }{\sum_{q=1}^J C_{\text{emiss}}(w_q,c_k)}
$$



In [43]:
initial_probs = (initial_counts / sum(initial_counts))

2-element Array{Float64,1}:
 0.333333
 0.666667

In [44]:
transition_probs = transition_counts./(sum(transition_counts, 2) + final_counts)

2×2 Array{Float64,2}:
 0.625  0.0
 0.5    0.5

In [53]:
final_probs =  final_counts ./ (sum(transition_counts, 2) + final_counts )

2×1 Array{Float64,2}:
 0.375
 0.0  

In [61]:
final_probs =  [final_probs[1], final_probs[2]]

2-element Array{Float64,1}:
 0.375
 0.0  

In [62]:
# notice the sum per row sums to one
[transition_probs final_probs]

2×3 Array{Float64,2}:
 0.625  0.0  0.375
 0.5    0.5  0.0  

In [63]:
# notice the sum per row sums to one
emission_probs = (emission_counts ./ sum(emission_counts, 2))

2×4 Array{Float64,2}:
 0.0  0.25  0.375  0.375
 0.0  0.75  0.0    0.25 

In [64]:
state_to_pos

Dict{String,Int64} with 2 entries:
  "sunny" => 1
  "rainy" => 2

In [65]:
word_to_pos

Dict{String,Int64} with 4 entries:
  "tennis" => 1
  "walk"   => 2
  "clean"  => 3
  "shop"   => 4

## Defining an HMM

In [66]:
typeof(initial_counts)

Array{Float64,1}

In [67]:
immutable Hmm
    possible_words::Set{String}
    possible_tags::Set{String}
    
    word_to_pos::Dict{String, Int64}
    state_to_pos::Dict{String, Int64}
    
    initial_counts::Vector{Int64}
    transition_counts::Matrix{Int64} 
    emission_counts::Matrix{Int64}
    final_counts::Vector{Int64}

    initial_probs::Vector{Float64}
    transition_probs::Matrix{Float64}
    emission_probs::Matrix{Float64}
    final_probs::Vector{Float64}
end

In [70]:
hmm =   Hmm(possible_words,
            possible_states,
            word_to_pos,
            state_to_pos,
            initial_counts,
            transition_counts,
            emission_counts,
            final_counts,
            initial_probs,
            transition_probs,
            emission_probs,            
            final_probs)

Hmm(Set(String["tennis","walk","clean","shop"]),Set(String["sunny","rainy"]),Dict("tennis"=>1,"walk"=>2,"clean"=>3,"shop"=>4),Dict("sunny"=>1,"rainy"=>2),[1,2],[5 0; 2 2],[0 2 3 3; 0 3 0 1],[3,0],[0.333333,0.666667],[0.625 0.0; 0.5 0.5],[0.0 0.25 0.375 0.375; 0.0 0.75 0.0 0.25],[0.375,0.0])

## Computing scores for a given sequence


For convenience, we will be working with 
log-probabilities, rather than probabilities. Therefore, if we associate to each circle and arrow in the trellis a score that corresponds
to the log-probabilities above, and if we define the score of a path
connecting the ${\tt start}$ and  ${\tt stop}$ symbols as
the sum of the scores of the circles and arrows it traverses, 
then the goal of **finding the most likely sequence of states (Viterbi decoding) corresponds to finding the path with the highest score**.



The trellis scores are given by the following expressions:

- For each state $c_k$:

\begin{eqnarray}
\mathrm{score}_{\mathrm{init}}(c_k) &=&
\log P_{\mathrm{init}}(Y_{1} = c_k | \text{start}).
\end{eqnarray}


- For each position $i \in {1,\ldots,N-1}$ and each pair of states $c_k$ and $c_l$:

\begin{eqnarray}
\mathrm{score}_{\mathrm{trans}}(i, c_k, c_l) &=&
\log P_{\mathrm{trans}}(Y_{i+1} = c_k | Y_i = c_l).
\end{eqnarray}


- For each state $c_l$:

\begin{eqnarray}
\mathrm{score}_{\mathrm{final}}(c_l) &=&
\log P_{\mathrm{final}}(\text{stop} | Y_N = c_l).
\end{eqnarray}


- For each position $i \in {1,\ldots,N}$ and state $c_k$:

\begin{eqnarray}
\mathrm{score}_{\mathrm{emiss}}(i, c_k) &=&
\log P_{\mathrm{emiss}}(X_i = x_i | Y_i = c_k).
\end{eqnarray}


Some python code for computin the scores

    def compute_scores(self, sequence):
        length = len(sequence.x) # Length of the sequence.
        num_states = self.get_num_states() # Number of states of the HMM.

        # Initial position.
        initial_scores = np.log(self.initial_probs)

        # Intermediate positions
        transition_scores = np.log(self.transition_probs) 
        emission_scores = np.log(self.emission_probs[sequence.x,:])

        # Final position.
        final_scores = np.log(self.final_probs)

        return initial_scores, transition_scores, final_scores, emission_scores

In [112]:
function compute_scores(hmm, sequence)
    length_sequence = length(sequence.words)
    n_states = length(hmm.possible_tags)
    
    initial_scores = log(hmm.initial_probs)
    transition_scores = log(hmm.transition_probs)

    sequence_words_integers = [word_to_pos[x] for x in sequence.words]
    emission_scores = log(hmm.emission_probs[:, sequence_words_integers])
    final_scores = log(hmm.final_probs)
    
    return initial_scores, transition_scores, final_scores, emission_scores
end

compute_scores (generic function with 1 method)

In [128]:
#log_likelihood, forward = hmm.decoder.run_forward(initial_scores, transition_scores,final_scores, emission_scores)
scores = compute_scores(hmm, train_sequences[1])


([-1.09861,-0.405465],
[-0.470004 -Inf; -0.693147 -0.693147],

[-0.980829,-Inf],
[-1.38629 -1.38629 -0.980829 -0.980829; -0.287682 -0.287682 -1.38629 -Inf])

In [129]:
scores

([-1.09861,-0.405465],
[-0.470004 -Inf; -0.693147 -0.693147],

[-0.980829,-Inf],
[-1.38629 -1.38629 -0.980829 -0.980829; -0.287682 -0.287682 -1.38629 -Inf])

## Posterior decoding


Posterior decoding consists
in picking state with the highest posterior for each position in the sequence independently; for 
each $i = 1,\ldots,N$:

\begin{equation}
y_i^* = \text{argmax}_{y_i \in \Lambda} P(Y_i=y_i | X = x).
\end{equation}

The **sequence posterior distribution** is the probability of a particular
hidden state sequence given that we have observed a particular
sequence. Moreover, we will be interested in two other posteriors distributions:
the **state posterior distribution**, corresponding to the
probability of being in a given state in a certain position given the
observed sequence; and the \textbf{transition posterior distribution},
which is the probability of making a particular transition, from position $i$ to
$i+1$, given the observed sequence. 

They are formally defined as follows:

- Sequence  Posterior
$$P(Y=y|X=x) = \frac{P(X=x,Y=y)}{P(X=x)}
$$

- State Posterior
$$
P(Y_i=y_i | X=x)
$$

- Transition Posterior
$$
P(Y_{i+1}=y_{i+1},Y_i=y_i| X=x)
$$


### Computing posteriors involves beeing able to compute $P(X=x)$
To compute the posteriors, a first step is to be able to compute the 
likelihood of
the sequence $P(X=x)$, which corresponds to summing the probability of all
possible hidden state sequences.

\begin{equation}
\mathbf{Likelihood\!:}\;\;\;\; P(X=x) = \displaystyle \sum_{y \in \Lambda^N} P(X=x,Y=y).
\end{equation}

The number of possible hidden state sequences is exponential in the
length of the sequence ($|\Lambda|^N$),
 which makes the sum over all of them hard. 
 In our simple
 example, there are $2^4 = 16$ paths, which we can actually explicitly enumerate
 and calculate their probability using Equation of the joint probability $P(x,y)$. But this is as far as it goes: for example, for Part-of-Speech
 tagging with a small tagset of 12 tags and a medium size
 sentence of length 10, there are $12^{10} = 61 917 364 224$ such
 paths. 
 
 Yet, we must be able to compute this sum (sum over $y \in \Lambda^N$) to compute the above likelihood
formula; this is called the inference problem. For sequence models, there is a well known dynamic programming algorithm,
the **Forward-Backward** (FB) algorithm, which allows the computation
to be performed in linear time, The runtime is linear with respect
to the sequence length. More precisely, 
the runtime is $O(N|\Lambda|^2)$. 
A naive enumeration would cost $O(|\Lambda|^N)$.

The FB algorithm relies on the independence of previous states
assumption, which  
is illustrated in the trellis view by having arrows only between consecutive states. 
The FB algorithm defines two auxiliary probabilities, the forward probability and the backward probability. 


## Efficient forward probability computation

The forward probability represents the probability that in position
$i$ we are in state $Y_i = c_k$ and that we have observed $x_1,\ldots,x_i$
up to that position. Therefore, its mathematical expression is:
\begin{equation}
\mathbf{Forward \ Probability\!:}\;\;\;\;  \mathrm{forward}(i, c_k) = P(Y_i = c_k, X_1=x_1,\ldots, X_i = x_i)
\end{equation}


Using the independence assumptions of the HMM we can compute $\mathrm{forward}(i, c_k)$ using all the forward computations \{$\mathrm{forward}(i -1, c)$ for $c \in \Lambda$\}. In order to facilitate the notation of the following argument we will denote by $x_{i:j}$  the assignemnt $X_i = x_i, \dots, X_j = x_j$. Therefore we can write   $\mathrm{forward}(i, y_i) $ as $P( y_i, x_{1:i } ) $ and rewrite the forward expression as follows:

\begin{equation}
  P( y_i, x_{1:i } ) =  \sum_{y_{i-1} \in \Lambda} P( y_i ,y_{i-1}, x_{1:i } )  =  \sum_{y_{i-1} \in \Lambda} P( x_i  | y_i,  y_{i-1},  x_{1:i-1 } ) \cdot P(y_i  | y_{i-1},  x_{1:i-1 }) \cdot P(y_{i-1},  x_{1:i-1 })  
\end{equation}


Using the **Observation independence** and the **Independence of previous states** properties of the first order HMM we have $P( x_i  | y_i,  y_{i-1},  x_{1:i-1 } ) = P( x_i  | y_i) $ and $P(y_i  | y_{i-1},  x_{1:i-1 })  = P(y_i  | y_{i-1})  $. Therefore the previous equation can be written, 
for $i \in \{2,\dots,N\}$ (where $N$ is the length of the sequence), as 

\begin{equation}
 \mathrm{forward}(i, y_i)  = \sum_{y_{i-1} \in \Lambda} P( x_i  | y_i, ) \cdot P(y_i  | y_{i-1}) \cdot \mathrm{forward}(i-1, y_{i-1})   
\end{equation}


The previous equation proves that  the forward probability can be defined by the
following recurrence rule: 

\begin{eqnarray}
\mathrm{forward}(1, c_k)&=& P_{\text{init}}(c_k|\text{start}) \times P_{\mathrm{emiss}}(x_1 | c_k)
 \\
 \mathrm{forward}(i, c_k) &=& \left(  \sum_{c_l \in \Lambda} P_{\mathrm{trans}}(c_k | c_l) \times \mathrm{forward}(i-1, c_l) \right) \times P_{\mathrm{emiss}}(x_i | c_k) 
 \\
  \mathrm{forward}(N+1, \text{stop}) &=& \sum_{c_l \in \Lambda} P_{\text{final}}(\text{ stop} | c_l) \times \mathrm{forward}(N, c_l).
\end{eqnarray}


Using the forward trellis one can compute the likelihood simply as:

\begin{equation}
P(X=x) = \mathrm{forward}(N+1, \text{ stop}).
\end{equation}

Although the forward probability is enough to calculate the likelihood of a given sequence, we will also need the backward probability to calculate the state posteriors. 

## Efficient backward probability computation



The backward probability is similar to the forward probability, but operates in the inverse direction.
It represents the probability of observing $x_{i+1},\ldots,x_N$ from position $i+1$ up to $N$, given that at position $i$ we are at state $Y_i = c_l$:

\begin{equation}
\mathbf{Backward \ Probability\!:}\;\;\;\;  \text{backward}(i, c_l) = P(X_{i+1}=x_{i+1},\ldots, X_N=x_N | Y_i = c_l).
\end{equation}



Using the independence assumptions of the HMM we can compute $\text{backward}(i, c_k)$ using all the backward computations $\text{backward}(i +1, c)$ for $c \in \Lambda$.

Therefore we can write   $\text{backward}(i, y_i) $ as $P( x_{i+1:N} | y_i ) $ and rewrite the forward expression as follows:

\begin{equation}
  P( x_{i+1:N} | y_i ) =  \sum_{y_{i+1} \in \Lambda} P( x_{i+1:N}, y_{i+1} | y_i)  =  \sum_{y_{i+1} \in \Lambda} P( x_{i+2:N} | y_i, y_{i+1}, x_{i+1}) 
   P( x_{i+1}, |  y_{i+1},  y_{i}) P( y_{i+1} | y_i)
\end{equation}

Using the previous equation we have proved that the backward probability can be defined by the following recurrence rule:


\begin{eqnarray}
\mathrm{backward}(N, c_l) &=& P_{\text{final}}(\text{stop} | c_l)  \\
\text{backward}(i, c_l) &=&  \displaystyle \sum_{c_k \in \Lambda} P_{\text{trans}}(c_k | c_l) \times 
\text{backward}(i+1, c_k) \times P_{\text{emiss}}(x_{i+1} | c_k) 
 \\
  \mathrm{backward}(0, \text{start}) &=& \sum_{c_k \in \Lambda} P_{\mathrm{init}}(c_k | \text{ start}) \times \mathrm{backward}(1, c_k) \times P_{\mathrm{emiss}}(x_{1} | c_k).
 \end{eqnarray}

Using the backward trellis one can compute the likelihood simply as:

\begin{equation}
P(X=x) = \mathrm{backward}(0, \text{start}).
\end{equation}


## The forward backward algorithm

We have seen how we can compute the probability of a sequence $x$ using the the forward and backward probabilities by computing  $\mathrm{forward}(N+1, \text{ stop})$ and $ \mathrm{backward}(0, \text{ start})$ respectively. Moreover,  the probability of a sequence $x$ can be computed with both forward and backward probabilities at a particular position $i$. 

The probability of a  given sequence $x$ at any position $i$ in the sequence can be computed
as follows:


\begin{eqnarray}
  P(X=x) &=& 
  \sum_{c_k \in \Lambda} P(X_1=x_1,\ldots, X_N=x_N,Y_i=c_k)\nonumber\\
  & =&
  \sum_{c_k \in \Lambda} 
  \underbrace{P(X_1=x_1,\ldots, X_i=x_i, Y_i=c_k)}_{\mathrm{forward}(i,c_k)} \times 
  \underbrace{P(X_{i+1}=x_{i+1},\ldots, X_N=x_N| Y_i=c_k)}_{\mathrm{backward}(i,c_k)}\nonumber\\
  &=& \sum_{c_k \in \Lambda} \mathrm{forward}(i,c_k) \times \mathrm{backward}(i,c_k).
\end{eqnarray}



This equation will work for any choice of $i$. Although redundant, this fact is useful when implementing an
HMM as a sanity check that the computations are being performed
correctly, since one can compute this expression for several $i$; they should all yield the same value. 

The following pseudocode shows the the forward backward algorithm. 

<img src="./images_for_notebooks/day_2/fb_alg.png"  style="max-width:100%; width: 50%">

The reader can notice that the $forward$ and $backward$ computations in the algorithm make use of $P_{emiss}$ and $P_{trans}$. There are a couple of details that should be taken into account if the reader wants to understand the algorithm using scores instead of probabilities.


- $forward(i,\hat{c})$  is computed using $P_{emiss}(x_i | \hat{c})$ which does not depend on the sum over all possible states $c_k \in  \Lambda $. Therefore when taking the logarithm of the sum over all possible states the recurrence of the forward computations can be split as a sum of two logarithms.


- $backward(i,\hat{c})$  is computed using $ P_{\text{trans}}(c_k | \hat{c} )$ and $P_{\text{emiss}}(x_{i+1} | c_k) $ both of  which  depend on $c_k$. Therefore when taking the logarithm of the sum the expression cannot be split as a sum of logarithms.



Given the forward and backward probabilities, one can compute both the state
and transition posteriors as follows:


\begin{align}
 \mathbf{State \ Posterior\!:}\;\;\;\;  & P(Y_i = y_i| X=x) = \frac{\mathrm{forward}(i, y_i) \times 
 \mathrm{backward}(i, y_i)}{P(X=x)}\\
 \mathbf{Transition \ Posterior\!:}\;\;\;\; &
 P(Y_i = y_i, Y_{i+1} = y_{i+1} | X=x)= \nonumber\\
 &
   \frac{\mathrm{forward}(i, y_i) \times 
   P_{\mathrm{trans}}(y_{i+1}|y_i) \times
   P_{\mathrm{emiss}}(x_{i+1}|y_{i+1}) \times
 \mathrm{backward}(i+1, y_{i+1})}{P(X=x)}
\end{align}

As a practical example, given that the person performs the sequence of actions $\text{ walk} \text{ walk} \text{ shop} \text{ clean}$, we want to know the probability of having been raining in the second day. The state posterior probability for this event can be seen as the probability that the sequence of actions above was generated by a sequence of weathers and where it was raining in the second day. In this case, the possible sequences would be all the sequences which have {\tt rainy} in the second position.


Using the state posteriors, we are ready to perform posterior
decoding. 
The strategy is to compute the state posteriors 
for each position $i \in \{1,\ldots,N\}$
and each state $c_k \in \Lambda$, and 
then pick the arg-max at each position:

$$
{\widehat y_i} := \text{argmax}_{y_i \in \Lambda} P(Y_i=y_i| X=x).
$$


#### Forward


        def run_forward(self, 
                        initial_scores,
                        transition_scores,
                        final_scores,
                        emission_scores):
                length = np.size(emission_scores, 0) 
                num_states = np.size(initial_scores) # Number of states.

                # Forward variables.
                forward = np.zeros([length, num_states]) + logzero()

                # Initialization.
                forward[0,:] = emission_scores[0,:] + initial_scores

                # Forward loop.
                for pos in xrange(1,length):
                    for current_state in xrange(num_states):
                        # x log domain turns a sum and sum turns a logsum
                        forward[pos, current_state] = \
                                logsum(forward[pos-1, :] +
                                transition_scores[pos-1, current_state, :])
                        forward[pos, current_state] += emission_scores[pos, current_state]

                # Termination.
                log_likelihood = logsum(forward[length-1,:] + final_scores)

                return log_likelihood, forward

In [ ]:
#initial_scores, transition_scores, final_scores, emission_scores = hmm.compute_scores(
#                                                                   simple.train.seq_list[0])

## Viterbi decoding